In [1]:
import pandas as pd

In [2]:
stops = pd.read_csv("../../data/raw/S_VISTA_1220_SA1.csv")
stops = stops[stops["wdstopwgt_sa3"].notna()]

C:\Users\dlaa0001\AppData\Local\Temp\ipykernel_9268\2794316435.py:1: DtypeWarning: Columns (46,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  stops = pd.read_csv("../../data/raw/S_VISTA_1220_SA1.csv")


In [3]:
stops.columns

Index(['stopid', 'persid', 'hhid', 'stopno', 'starthour', 'startime',
       'arrhour', 'arrtime', 'duration', 'dephour', 'deptime', 'travtime',
       'vistadist', 'origplace1', 'origplace2', 'origpurp1', 'origpurp2',
       'destplace1', 'destplace2', 'destpurp1', 'destpurp2', 'origsa1',
       'origsa2', 'origsa2_name', 'origsa3', 'origsa3_name', 'origsa4',
       'origsa4_name', 'origlga', 'destsa1', 'destsa2', 'destsa2_name',
       'destsa3', 'destsa3_name', 'destsa4', 'destsa4_name', 'destlga',
       'fullmode', 'mainmode', 'person1', 'person2', 'person3', 'person4',
       'person5', 'person6', 'person7', 'person8', 'person9', 'person10',
       'person11', 'wdstopwgt_sa3', 'westopwgt_sa3', 'wdstopwgt_lga',
       'westopwgt_lga'],
      dtype='object')

In [4]:
chosen_cols = ["persid", "stopid", "hhid", "stopno", "origplace1", "destplace1", "origpurp1", "destpurp1", "startime", "arrtime", "duration", "travtime"]
sub_stops = stops[chosen_cols]
sub_stops

,persid,stopid,hhid,stopno,origplace1,destplace1,origpurp1,destpurp1,startime,arrtime,duration,travtime
0,Y12H0000101P01,Y12H0000101P01S01,Y12H0000101,1,Accommodation,Workplace,At Home,Work Related,525,545,505.0,20
1,Y12H0000101P01,Y12H0000101P01S02,Y12H0000101,2,Workplace,Accommodation,Work Related,At or Go Home,1050,1075,NaN,25
2,Y12H0000101P02,Y12H0000101P02S01,Y12H0000101,1,Accommodation,Place of Education,At Home,Pick-up or Drop-off Someone,500,520,20.0,20
3,Y12H0000101P02,Y12H0000101P02S02,Y12H0000101,2,Place of Education,Shops,Pick-up or Drop-off Someone,Buy Something,540,550,50.0,10
4,Y12H0000101P02,Y12H0000101P02S03,Y12H0000101,3,Shops,Accommodation,Buy Something,At or Go Home,600,610,280.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...
247820,Y20H4001029P02,Y20H4001029P02S01,Y20H4001029,1,Accommodation,Place of Personal Business,At Home,Personal Business,600,620,5.0,20
247821,Y20H4001029P02,Y20H4001029P02S02,Y20H4001029,2,Place of Personal Business,Accommodation,Personal Business,At or Go Home,625,640,NaN,15
247822,Y20H4001039P01,Y20H4001039P01S01,Y20H4001039,1,Accommodation,Place of Personal Business,At Home,Personal Business,540,550,40.0,10
247823,Y20H4001039P01,Y20H4001039P01S02,Y20H4001039,2,Place of Personal Business,Place of Personal Business,Personal Business,Personal Business,590,650,70.0,60


In [5]:
have_first_stop = sub_stops.groupby("persid")["stopno"].apply(lambda x: list(x)).apply(lambda x: 1 in x)[lambda x: x]
sub_stops = sub_stops[sub_stops["persid"].isin(have_first_stop.index)]

In [6]:
# Group by persid and get first/last locations
first_las_loc_check = (
    sub_stops
    .sort_values("stopno")
    .groupby("persid")
    .agg({
        "origplace1": "first",
        "destplace1": "last"
    })
    .reset_index()
)
home_bound_pp = first_las_loc_check[(first_las_loc_check["origplace1"] == "Accommodation") & (first_las_loc_check["destplace1"] == "Accommodation")]
assert home_bound_pp["origplace1"].nunique() == home_bound_pp["destplace1"].nunique() == 1

In [7]:
final_stops = sub_stops[sub_stops["persid"].isin(home_bound_pp["persid"])]
final_stops["destpurp1"].value_counts()

destpurp1
At or Go Home                   61679
Change Mode                     31644
Work Related                    24857
Buy Something                   13953
Pick-up or Drop-off Someone     12463
Social                          12272
Personal Business                7852
Education                        7758
Recreational                     7445
Accompany Someone                5664
Pick-up or Deliver Something     2183
Other Purpose                     396
Not Stated                          4
Unknown Purpose                     1
Name: count, dtype: int64

In [8]:
purpose_groups = {
    # Home & Transit
    "At or Go Home": "Home",
    "Change Mode": "Transit",

    # Work & Education
    "Work Related": "Work",
    "Education": "Education",

    # Shopping
    "Buy Something": "Shopping",
    "Personal Business": "Shopping",
    "Pick-up or Deliver Something": "Shopping",

    # Social
    "Social": "Social",
    "Recreational": "Social",

    # Accompanying
    "Pick-up or Drop-off Someone": "Accompanying",
    "Accompany Someone": "Accompanying",

    # Other
    "Other Purpose": "Other",
    "Not Stated": "Other",
    "Unknown Purpose": "Other"
}
final_stops["purpose"] = final_stops["destpurp1"].map(purpose_groups)
final_stops["purpose"].value_counts()

C:\Users\dlaa0001\AppData\Local\Temp\ipykernel_9268\3519503332.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_stops["purpose"] = final_stops["destpurp1"].map(purpose_groups)


purpose
Home            61679
Transit         31644
Work            24857
Shopping        23988
Social          19717
Accompanying    18127
Education        7758
Other             401
Name: count, dtype: int64

In [9]:
# Get the last stop for each person
last_stops = final_stops.groupby("persid")["stopno"].max().reset_index()
last_stop_check = final_stops.merge(last_stops, on=["persid", "stopno"])
# Assert that duration is NaN for all last stops
# assert last_stop_check["duration"].isna().all(), "Some last stops have non-NaN duration values"
to_rm_pp = last_stop_check[(last_stop_check["stopno"] == 1) | (last_stop_check["purpose"] != "Home")]["persid"]
final_stops = final_stops[~final_stops["persid"].isin(to_rm_pp)]
last_stop_check = last_stop_check[~last_stop_check["persid"].isin(to_rm_pp)]
final_stops["purpose"].value_counts()

purpose
Home            61353
Transit         31170
Work            24658
Shopping        23133
Social          19238
Accompanying    17959
Education        7669
Other             386
Name: count, dtype: int64

In [10]:
import numpy as np
# Set duration to NaN for last stops
if final_stops.index.name != "stopid":
    final_stops = final_stops.set_index("stopid")
final_stops.loc[last_stop_check["stopid"], "duration"] = np.nan

# 1440 represents end of day (24 hours * 60 minutes)
final_stops["duration"] = final_stops["duration"].fillna(1440 - final_stops["arrtime"])

# Assert no negative durations
# assert (final_stops["duration"] >= 0).all(), "Found negative duration values"
# We will remove any negative durations
final_stops = final_stops[final_stops["duration"] >= 0]

In [11]:
final_stops["total_duration"] = final_stops["duration"] + final_stops["travtime"]
final_stops

,persid,hhid,stopno,origplace1,destplace1,origpurp1,destpurp1,startime,arrtime,duration,travtime,purpose,total_duration
stopid,,,,,,,,,,,,,
Y12H0000101P01S01,Y12H0000101P01,Y12H0000101,1,Accommodation,Workplace,At Home,Work Related,525,545,505.0,20,Work,525.0
Y12H0000101P01S02,Y12H0000101P01,Y12H0000101,2,Workplace,Accommodation,Work Related,At or Go Home,1050,1075,365.0,25,Home,390.0
Y12H0000101P02S01,Y12H0000101P02,Y12H0000101,1,Accommodation,Place of Education,At Home,Pick-up or Drop-off Someone,500,520,20.0,20,Accompanying,40.0
Y12H0000101P02S02,Y12H0000101P02,Y12H0000101,2,Place of Education,Shops,Pick-up or Drop-off Someone,Buy Something,540,550,50.0,10,Shopping,60.0
Y12H0000101P02S03,Y12H0000101P02,Y12H0000101,3,Shops,Accommodation,Buy Something,At or Go Home,600,610,280.0,10,Home,290.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Y20H4001029P02S01,Y20H4001029P02,Y20H4001029,1,Accommodation,Place of Personal Business,At Home,Personal Business,600,620,5.0,20,Shopping,25.0
Y20H4001029P02S02,Y20H4001029P02,Y20H4001029,2,Place of Personal Business,Accommodation,Personal Business,At or Go Home,625,640,800.0,15,Home,815.0
Y20H4001039P01S01,Y20H4001039P01,Y20H4001039,1,Accommodation,Place of Personal Business,At Home,Personal Business,540,550,40.0,10,Shopping,50.0


In [12]:
filter_final_stops = final_stops[["persid", "hhid", "stopno", "purpose", "startime", "total_duration"]]
filter_final_stops

,persid,hhid,stopno,purpose,startime,total_duration
stopid,,,,,,
Y12H0000101P01S01,Y12H0000101P01,Y12H0000101,1,Work,525,525.0
Y12H0000101P01S02,Y12H0000101P01,Y12H0000101,2,Home,1050,390.0
Y12H0000101P02S01,Y12H0000101P02,Y12H0000101,1,Accompanying,500,40.0
Y12H0000101P02S02,Y12H0000101P02,Y12H0000101,2,Shopping,540,60.0
Y12H0000101P02S03,Y12H0000101P02,Y12H0000101,3,Home,600,290.0
...,...,...,...,...,...,...
Y20H4001029P02S01,Y20H4001029P02,Y20H4001029,1,Shopping,600,25.0
Y20H4001029P02S02,Y20H4001029P02,Y20H4001029,2,Home,625,815.0
Y20H4001039P01S01,Y20H4001039P01,Y20H4001039,1,Shopping,540,50.0


In [13]:
# Get first stop for each person to calculate initial duration
first_stops = filter_final_stops[filter_final_stops["stopno"] == 1].copy()

# Create stopno 0 entries
zero_stops = first_stops.copy()
zero_stops["stopno"] = 0
zero_stops["purpose"] = "Home"
zero_stops["startime"] = 0
zero_stops["total_duration"] = first_stops["startime"]

# Combine with original stops and sort
filter_final_stops = pd.concat([zero_stops, filter_final_stops])
filter_final_stops = filter_final_stops.sort_values(["persid", "stopno"]).reset_index(drop=True)
filter_final_stops

,persid,hhid,stopno,purpose,startime,total_duration
0,Y12H0000101P01,Y12H0000101,0,Home,0,525.0
1,Y12H0000101P01,Y12H0000101,1,Work,525,525.0
2,Y12H0000101P01,Y12H0000101,2,Home,1050,390.0
3,Y12H0000101P02,Y12H0000101,0,Home,0,500.0
4,Y12H0000101P02,Y12H0000101,1,Accompanying,500,40.0
...,...,...,...,...,...,...
228565,Y20H4001029P02,Y20H4001029,2,Home,625,815.0
228566,Y20H4001039P01,Y20H4001039,0,Home,0,540.0
228567,Y20H4001039P01,Y20H4001039,1,Shopping,540,50.0
228568,Y20H4001039P01,Y20H4001039,2,Shopping,590,130.0


In [14]:
filter_final_stops["persid"].value_counts()

persid
Y19H3612401P01    29
Y15H1570503P01    25
Y16H1990323P02    25
Y19H3840302P02    24
Y19H3530411P01    24
                  ..
Y13H0260305P01     2
Y12H0170339P02     2
Y12H0150222P01     2
Y12H0130523P02     2
Y12H0070109P01     2
Name: count, Length: 43453, dtype: int64

### Update to remove not matching cases

In [15]:
check_stops_total_duration = filter_final_stops.groupby("persid")["total_duration"].sum()
check_stops_total_duration = check_stops_total_duration[check_stops_total_duration != 1440]
check_stops_total_duration

persid
Y12H0010530P01    1365.0
Y12H0020222P01    1505.0
Y12H0040119P02    1451.0
Y12H0040119P03    1451.0
Y12H0070109P01    1480.0
                   ...  
Y20H3910526P01    1515.0
Y20H3920226P01    1500.0
Y20H3920233P01    1455.0
Y20H3940305P04    1450.0
Y20H3940336P01    1500.0
Name: total_duration, Length: 555, dtype: float64

In [16]:
filter_final_stops = filter_final_stops[~filter_final_stops["persid"].isin(check_stops_total_duration.index)]
filter_final_stops

,persid,hhid,stopno,purpose,startime,total_duration
0,Y12H0000101P01,Y12H0000101,0,Home,0,525.0
1,Y12H0000101P01,Y12H0000101,1,Work,525,525.0
2,Y12H0000101P01,Y12H0000101,2,Home,1050,390.0
3,Y12H0000101P02,Y12H0000101,0,Home,0,500.0
4,Y12H0000101P02,Y12H0000101,1,Accompanying,500,40.0
...,...,...,...,...,...,...
228565,Y20H4001029P02,Y20H4001029,2,Home,625,815.0
228566,Y20H4001039P01,Y20H4001039,0,Home,0,540.0
228567,Y20H4001039P01,Y20H4001039,1,Shopping,540,50.0
228568,Y20H4001039P01,Y20H4001039,2,Shopping,590,130.0


In [17]:
curr_stop_no = filter_final_stops["stopno"].to_list()
fixed_stop_no = []
prev_stop = None
for stop_no in curr_stop_no:
    if stop_no == 0:
        prev_stop = 0
    else:
        prev_stop += 1
    fixed_stop_no.append(prev_stop)
filter_final_stops["stopno"] = fixed_stop_no

C:\Users\dlaa0001\AppData\Local\Temp\ipykernel_9268\3504837752.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_final_stops["stopno"] = fixed_stop_no


In [18]:
# Check that stopno is contiguous
check_stop_no = filter_final_stops.groupby("persid")["stopno"].apply(list)
for index, i in check_stop_no.items():
    check = [x for x in range(len(i))]
    assert i == check, f"Mismatch found: {i} != {check} of {index}"

## Remove Transit

In [19]:

def fold_transit_for_person(pdf: pd.DataFrame) -> pd.DataFrame:
    """
    pdf: rows for one person, expected columns:
        persid, hhid, stopno, purpose, startime, total_duration
    Assumes rows cover a continuous day timeline.
    """
    pdf = pdf.sort_values("stopno").reset_index(drop=True)

    out_rows = []
    carry = 0.0          # minutes of transit to fold into the next non-Transit stop
    carry_start = None   # the start time of the first transit in a contiguous block

    for i, row in pdf.iterrows():
        purpose = str(row["purpose"])
        startime = float(row["startime"])
        dur = float(row["total_duration"])

        if purpose == "Transit":
            # Start a (possibly) contiguous transit block.
            if carry == 0:
                carry_start = startime
            carry += dur
            continue

        # Non-transit row: apply any carried transit
        if carry > 0:
            # Move the start back to the first transit start,
            # and extend duration by the whole transit block
            new_start = carry_start
            new_dur = dur + carry
            carry = 0.0
            carry_start = None
        else:
            new_start = startime
            new_dur = dur

        new_row = row.copy()
        new_row["startime"] = new_start
        new_row["total_duration"] = new_dur
        out_rows.append(new_row)

    # Note: If the last rows were Transit, there is no “following” activity to receive it.
    # In typical home-bound days this should not occur, but if it does we simply drop the
    # trailing Transit (no receiver). You can alternatively add it to the last non-Transit:
    # if carry > 0 and out_rows:
    #     out_rows[-1]["total_duration"] += carry

    out = pd.DataFrame(out_rows)

    # Recompute stopno sequentially from 0 for this person
    out = out.sort_values("startime", kind="mergesort").reset_index(drop=True)
    out["stopno"] = range(len(out))

    # Optional integrity checks (comment out if not needed):
    # assert out["startime"].is_monotonic_increasing
    # timeline_ok = (out["startime"] + out["total_duration"]).round(6).shift(1).fillna(out.loc[0, "startime"]) \
    #               .equals(out["startime"])
    # if not timeline_ok: print("Warning: timeline discontinuity for persid", out.loc[0, "persid"])

    return out

# Process per person
filter_final_stops = (
    filter_final_stops.groupby("persid", group_keys=False)
        .apply(fold_transit_for_person)
        .reset_index(drop=True)
)

C:\Users\dlaa0001\AppData\Local\Temp\ipykernel_9268\3641852697.py:65: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(fold_transit_for_person)


In [20]:
# Remove duplicate consecutive activities
def fold_duplicates(group):
    group = group.sort_values("stopno").reset_index(drop=True)
    
    new_rows = []
    prev_purpose = None
    acc_duration = 0
    start_time = None
    stop_counter = 0
    
    for _, row in group.iterrows():
        if prev_purpose is None:
            # First activity
            prev_purpose = row["purpose"]
            acc_duration = row["total_duration"]
            start_time = row["startime"]
        elif row["purpose"] == prev_purpose:
            # Same as previous -> accumulate duration
            acc_duration += row["total_duration"]
        else:
            # Save the previous block
            new_rows.append({
                "persid": row["persid"],
                "hhid": row["hhid"],
                "stopno": stop_counter,
                "purpose": prev_purpose,
                "startime": start_time,
                "total_duration": acc_duration
            })
            stop_counter += 1
            
            # Start a new block
            prev_purpose = row["purpose"]
            acc_duration = row["total_duration"]
            start_time = row["startime"]
    
    # Add the last block
    new_rows.append({
        "persid": group.iloc[-1]["persid"],
        "hhid": group.iloc[-1]["hhid"],
        "stopno": stop_counter,
        "purpose": prev_purpose,
        "startime": start_time,
        "total_duration": acc_duration
    })
    
    return pd.DataFrame(new_rows)

# Apply per person
filter_final_stops = filter_final_stops.groupby("persid", group_keys=False).apply(fold_duplicates).reset_index(drop=True)

C:\Users\dlaa0001\AppData\Local\Temp\ipykernel_9268\1241880715.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filter_final_stops = filter_final_stops.groupby("persid", group_keys=False).apply(fold_duplicates).reset_index(drop=True)


In [21]:
def has_consecutive_duplicates(purposes):
    return any(purposes[i] == purposes[i+1] for i in range(len(purposes)-1))

# Find persons with duplicate consecutive activities
dup_cases = []
for pid, g in filter_final_stops.groupby("persid"):
    g = g.sort_values("stopno")
    purposes = g["purpose"].tolist()
    if has_consecutive_duplicates(purposes):
        dup_cases.append((pid, purposes))

# Convert to DataFrame for inspection
dup_df = pd.DataFrame(dup_cases, columns=["persid", "purposes"])

assert len(dup_df) == 0, "have duplicate consecutive activities"

## Filter to get person

In [22]:
persons_atts = pd.read_csv("../../data/raw/P_VISTA_1220_SA1.csv")
persons_atts.columns

Index(['persid', 'hhid', 'persno', 'numstops', 'monthofbirth', 'yearofbirth',
       'age', 'sex', 'relationship', 'persinc', 'carlicence', 'mbikelicence',
       'otherlicence', 'nolicence', 'fulltimework', 'parttimework',
       'casualwork', 'anywork', 'studying', 'activities', 'mainact',
       'worktype', 'emptype', 'anzsco1', 'anzsco2', 'anzsic1', 'anzsic2',
       'startplace', 'additionaltravel', 'cycledwork', 'cycledshopping',
       'cycledexercise', 'cycledother', 'nocycled', 'wdperswgt_sa3',
       'weperswgt_sa3', 'wdperswgt_lga', 'weperswgt_lga'],
      dtype='object')

In [23]:
chosen_persons_cols = ["persid", "hhid", "age", "sex", "persinc", "nolicence", 'fulltimework', 'parttimework', 'casualwork', 'anywork', 'studying', "worktype"]
sub_persons = persons_atts[chosen_persons_cols]
sub_persons = sub_persons[sub_persons["persid"].isin(filter_final_stops["persid"])]
sub_persons

,persid,hhid,age,sex,persinc,nolicence,fulltimework,parttimework,casualwork,anywork,studying,worktype
0,Y12H0000101P01,Y12H0000101,50,M,$1000-1249 p.w.,Some Licence,Y,N,N,Y,No Study,Fixed Hours
1,Y12H0000101P02,Y12H0000101,43,F,$1-199 p.w.,Some Licence,N,N,N,N,No Study,Not in Work Force
2,Y12H0000101P03,Y12H0000101,11,F,Zero Income,No Licence,N,N,N,N,Primary,Not in Work Force
3,Y12H0000101P04,Y12H0000101,6,F,Zero Income,No Licence,N,N,N,N,Primary,Not in Work Force
5,Y12H0000102P02,Y12H0000102,56,M,$300-399 p.w.,Some Licence,N,N,N,N,No Study,Not in Work Force
...,...,...,...,...,...,...,...,...,...,...,...,...
78964,Y20H4001009P01,Y20H4001009,59,F,$400-599 p.w.,Some Licence,N,Y,N,Y,No Study,Fixed Hours
78965,Y20H4001009P02,Y20H4001009,57,M,$1000-1249 p.w.,Some Licence,Y,N,N,Y,No Study,Fixed Hours
78966,Y20H4001009P03,Y20H4001009,25,F,$1250-1499 p.w.,Some Licence,Y,N,N,Y,No Study,Fixed Hours
78975,Y20H4001029P02,Y20H4001029,70,F,$300-399 p.w.,Some Licence,N,N,N,N,No Study,Not in Work Force


# Output

In [ ]:
filter_final_stops.to_csv("../../data/24h_full_activities_homebound_wd.csv", index=False)
sub_persons.to_csv("../../data/24h_full_persons_homebound_wd.csv", index=False)

In [ ]:
n_tot_pp = len(sub_persons)
print(n_tot_pp)

In [25]:
# Sample by 0.1 to 0.9
for frac in np.arange(0.1, 1.0, 0.1):
    samp_persons = sub_persons.sample(frac=frac, random_state=42)
    samp_stops = filter_final_stops[filter_final_stops["persid"].isin(samp_persons["persid"])]
    
    samp_persons.to_csv(f"../../data/24h_{frac}_persons_homebound_wd.csv", index=False)
    samp_stops.to_csv(f"../../data/24h_{frac}_activities_homebound_wd.csv", index=False)

In [26]:
small_set_persons = sub_persons.sample(frac=0.05)
small_set_stops = filter_final_stops[filter_final_stops["persid"].isin(small_set_persons["persid"])]
small_set_stops.to_csv("../../data/24h_0.05_activities_homebound_wd.csv", index=False)
small_set_persons.to_csv("../../data/24h_0.05_persons_homebound_wd.csv", index=False)